In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data=pd.read_csv('D:\BIGDATAGISA-main\BIGDATAGISA-main/bikeshare.csv')
from sklearn.model_selection import train_test_split
X=data.drop(['count'], axis=1)
Y=data[['count']]
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.3, random_state=0)

#시험환경 : X_train, X_test, y_train까지 주어짐

In [3]:
train_data=pd.concat([X_train, y_train], axis=1)

In [4]:
train_data['datetime']=pd.to_datetime(train_data['datetime'])
X_test['datetime']=pd.to_datetime(X_test['datetime'])
pre_train_data_1=train_data[['datetime']]
pre_train_data_2=train_data[['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered']]
pre_train_data_3=train_data.drop(['datetime','temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered'], axis=1)
pre_test_data_1=X_test[['datetime']]
pre_test_data_2=X_test[['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered']]
pre_test_data_3=X_test.drop(['datetime','temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered'], axis=1)

In [5]:
#날짜 변수 처리
def make_date(df2):
    df=df2.copy()
    df['year']=df['datetime'].map(lambda x : x.year)
    df['month']=df['datetime'].map(lambda x : x.month)
    df['day']=df['datetime'].map(lambda x : x.day)
    df['weekend']=df['datetime'].map(lambda x : x.dayofweek)
    df['hour']=df['datetime'].map(lambda x : x.hour)
    df['minute']=df['datetime'].map(lambda x : x.minute)
    return df

set_train_data_1=make_date(pre_train_data_1)
set_test_data_1=make_date(pre_test_data_1)
set_train_data_1.drop(['datetime'], axis=1, inplace=True)
set_test_data_1.drop(['datetime'], axis=1, inplace=True)

In [6]:
#이상치 제거
def cut_outlier(df2, columns):
    df=df2.copy()
    for column in columns:
        q1=df[column].quantile(.25)
        q3=df[column].quantile(.75)
        iqr=q3-q1
        low=q1-1.5*iqr
        high=q3+1.5*iqr
        df.loc[df[column]<low, column]=low
        df.loc[df[column]>high, column]=high
    return df

set_train_data_2=cut_outlier(pre_train_data_2, ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered'])
set_test_data_2=cut_outlier(pre_test_data_2, ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered'])

In [7]:
train_data=pd.concat([set_train_data_1, set_train_data_2, pre_train_data_3], axis=1)
test_data=pd.concat([set_test_data_1, set_test_data_2, pre_test_data_3], axis=1)

In [8]:
#더미변수화
train_data_dum=pd.get_dummies(train_data)
test_data_dum=pd.get_dummies(test_data)
final_train_data, final_test_data = train_data_dum.align(test_data_dum, join='left', axis=1, fill_value=0)
final_test_data.drop(['count'], axis=1, inplace=True)

In [9]:
X=final_train_data.drop(['count'], axis=1)
Y=final_train_data[['count']]

# 모델링

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

max_depth=list(range(2,6))
min_samples_leaf=[1,2,5]
min_samples_split=[2,4,6]
max_features=['auto', 'sqrt']
bootstrap=[True, False]
n_estimators=list(range(100,200))

param_rf={'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'max_features': max_features,
         'bootstrap': bootstrap, 'n_estimators': n_estimators}


rf_clf=RandomForestRegressor()


random_rf=RandomizedSearchCV(estimator=rf_clf, param_distributions=param_rf, n_iter=10, cv=3, scoring='neg_mean_squared_error', refit=True,
                             return_train_score=True)
random_rf.fit(X, Y)
result_rf=pd.DataFrame(random_rf.cv_results_)
result_rf=result_rf.loc[:, ['mean_test_score', 'params']]
result_rf=result_rf.sort_values('mean_test_score', ascending=False).reset_index()

In [11]:
final_score=result_rf['mean_test_score'][0]
print(final_score)
final_params=result_rf['params'][0]
print(final_params)

-565.1013152671459
{'n_estimators': 194, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'auto', 'max_depth': 5, 'bootstrap': True}


In [17]:
rf_reg=RandomForestRegressor(n_estimators=194, min_samples_split=6, min_samples_leaf=5, max_features='auto', max_depth=5, bootstrap=True)
rf_reg.fit(X, Y)
pred=rf_reg.predict(final_test_data)
pred=pred.reshape(-1,1)
result=pd.DataFrame(pred, index=final_test_data.index).rename(columns={0:'prediction'})
result

,prediction
6638,233.966601
7975,263.010981
5915,263.010981
8050,411.758283
5894,371.054153
...,...
5521,5.032051
10626,14.092363
8126,71.096547
1633,63.111782
